In [1]:
import wptools, wikipedia, re

def checkAllAlpha(test_str):
    for c in test_str:
        if not c.isalpha():
            return False
    return True

def removeBracketsData(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    skip3c = 0
    skip4c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == '{':
            skip3c += 1
        elif i == '<':
            skip4c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif i == '}'and skip2c > 0:
            skip3c -= 1
        elif i == '>'and skip2c > 0:
            skip4c -= 1
        elif skip1c == 0 and skip2c == 0 and skip3c == 0 and skip4c == 0 and i!=',':
            ret += i
    return ret.strip()

def removeHTML(test_str):
    ret = ''
    skip4c = 0
    for i in test_str:
        if i == '<':
            skip4c += 1
        elif i == '>' and skip4c > 0:
            skip4c -= 1
        elif skip4c == 0:
            ret += i
    return ret.strip()

def populate_data(company_name, company_data_dict):
    try:
        # company_data_dict = wptools.page(wikipedia.search(company_name)[0]).get_parse(show=False).data['infobox']
        # return company_data_dict
        comp_data_to_store = {'name': company_name, 'type': '', 'industry': '', 'revenue': -1, 'hq_location': '', 'num_employees': -1, 'founded': -1}

        if 'name' in company_data_dict.keys():

            comp_data_to_store['name'] = removeHTML(company_data_dict['name'])

        if 'type' in company_data_dict.keys():
            str_to_process = company_data_dict['type']
            list_of_tokens = re.findall(r'([^\[\]]*)', str_to_process)
            for token in list_of_tokens:
                if len(token)>0:
                    comp_data_to_store['type'] = token.split('|')[-1]
                    break

        if 'industry' in company_data_dict.keys():
            str_to_process = company_data_dict['industry']
            list_of_tokens = re.findall(r'([^\[\]]*)', str_to_process)
            for token in list_of_tokens:
                token = token.strip()
                if len(token)>0 and checkAllAlpha(token[0]):
                    for tok in token.split('|'):
                        if len(tok)>3:
                            comp_data_to_store['industry'] = comp_data_to_store['industry']+ removeBracketsData(tok)+','
            comp_data_to_store['industry'] = comp_data_to_store['industry'][:-1]

        if 'revenue' in company_data_dict.keys():
            str_to_process = company_data_dict['revenue']
            index = -1
            revenue = 0
            if 'million' in str_to_process:
                index = str_to_process.index('million')
                revenue = 1000000
            elif 'billion' in str_to_process:
                index = str_to_process.index('billion')
                revenue = 1000000000
            elif 'thousand' in str_to_process:
                index = str_to_process.index('thousand')
                revenue = 1000
            print(index)
            if revenue>0:
                num = ''
                l = index-1
                while str_to_process[l]==' ':
                    l-=1
                while str_to_process[l].isdigit() or str_to_process[l]=='.':
                    num = str_to_process[l] + num
                    l-=1
                comp_data_to_store['revenue'] = float(num) * revenue

        if 'hq_location' in company_data_dict.keys():
            str_to_process = company_data_dict['hq_location']
            list_of_tokens = re.findall(r'([^\[\]]*)', str_to_process)
            for token in list_of_tokens:
                token = token.strip()
                if len(token)>0 and token[0].isalpha():
                    for tok in token.split('|'):
                        if len(tok)>0:
                            comp_data_to_store['hq_location'] = comp_data_to_store['hq_location'] + tok + ' '
            comp_data_to_store['hq_location'] = comp_data_to_store['hq_location'].strip()

        if 'num_employees' in company_data_dict.keys():
            str_to_process = removeBracketsData(company_data_dict['num_employees'])
            n_emp = 0
            for c in str_to_process:
                if (not c.isdigit()) and n_emp>0:
                    break
                elif c.isdigit():
                    n_emp = n_emp*10 + int(c)
            comp_data_to_store['num_employees'] = n_emp

        if 'founded' in company_data_dict.keys():
            str_to_process = company_data_dict['founded']
            list_of_tokens = re.findall(r'\d{4}', str_to_process)
            for year in list_of_tokens:
                comp_data_to_store['founded'] = int(year)

        return comp_data_to_store
    except:
          return None

In [2]:
import pickle
with open('wiki_dict.pkl', 'rb') as f:
    x = pickle.load(f)
wiki_data_populated = {}
count = 0
for k, v in x.items():
    d = populate_data(k)
    if d is None :
        continue
    wiki_data_populated[k] = d
    wiki_data_file = open("company_wiki_infobox_data.pkl", "wb")
    pickle.dump(wiki_data_populated, wiki_data_file)
    wiki_data_file.close()
    count+=1
    if count%100==0:
        print(count)
# populate_data('Apple (company)')

en.wikipedia.org (parse) (24)7.ai
en.wikipedia.org (parse) 280 North, Inc.
en.wikipedia.org (parse) 2d3
en.wikipedia.org (parse) 33Across
en.wikipedia.org (parse) 3D Systems
en.wikipedia.org (parse) 3ality Technica
en.wikipedia.org (parse) 4D Inc
en.wikipedia.org (parse) 4INFO
en.wikipedia.org (imageinfo) File:4INFO Headquarters.jpg
en.wikipedia.org (parse) A10 Networks
en.wikipedia.org (parse) ACI Worldwide
en.wikipedia.org (imageinfo) File:aci-headquarters-office.jpg
en.wikipedia.org (parse) ADARA Networks
en.wikipedia.org (parse) APC by Schneider Electric
en.wikipedia.org (parse) ARCOS LLC
en.wikipedia.org (parse) ASG Technologies
en.wikipedia.org (parse) AT&T
en.wikipedia.org (imageinfo) File:AT&THQDallas.jpg
en.wikipedia.org (parse) AT&T Cybersecurity
en.wikipedia.org (parse) AWS Elemental
en.wikipedia.org (parse) Abacus Data Systems
en.wikipedia.org (parse) Abbott Informatics
en.wikipedia.org (parse) Accent R
en.wikipedia.org (parse) Accordent Technologies
en.wikipedia.org (parse

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


In [7]:
# import pickle
# import yfinance as yf
# with open('wiki_data_populated.pkl', 'rb') as f:
#     x = pickle.load(f)
# with open('company_url_dict_updated.pkl', 'rb') as f:
#     x2 = pickle.load(f)
# with open('companies_best_match_symbol.pkl', 'rb') as f:
#     companies_best_match_symbol = pickle.load(f)
# import pandas as pd
# row_list = []
# for k, v in x.items():
#     if 'data' in v.keys():
#         continue
#     v['index_name'] = k
#     if v['index_name'] in companies_best_match_symbol.keys():
#         v['stock_symbol'] = companies_best_match_symbol[v['index_name']]
#         try:
#             v['stock_price'] = yf.Ticker(v['stock_symbol']).history(period='1d').tail(1)['Close'].iloc[0]
#         except:
#             v['stock_price'] = 0
#     v['student_careers_url'] = x2[k]
#     row_list.append(v)
# df = pd.DataFrame(row_list)
# df['revenue'] = df['revenue'].astype('int64', copy=False)
# df['founded'] = df['founded'].astype('int64', copy=False)
# df['num_employees'] = df['num_employees'].astype('int64', copy=False)
# df.head()
# # df.to_csv('companies_wikipedia_data.csv', index=False)

'137,000'

In [33]:
# df.to_csv('companies_wikipedia_data.csv', index=False)
